In [2]:
import pandas as pd
import numpy as np
import re

In [6]:
train_df = pd.read_csv('train.csv')

In [8]:
test_df = pd.read_csv('test.csv')
eval_df = pd.read_csv('eval.csv')

In [9]:
goods_df = pd.read_csv('goods.csv')

In [10]:
train_df

,target,good_0,good_1,good_2,good_3,good_4,good_5,good_6,good_7,good_8,...,good_564,good_565,good_566,good_567,good_568,good_569,good_570,good_571,good_572,good_573
0,person_0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,person_1,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
2,person_2,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,person_3,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,person_4,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3015,person_3015,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3016,person_3016,2,0,0,1,0,0,1,0,0,...,0,1,0,0,0,0,0,1,0,2
3017,person_3017,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3018,person_3018,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


# ContentBased recomendations

In [12]:
%pip install gensim

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [14]:
from sklearn.metrics.pairwise import cosine_similarity
import gensim

# form Content 

In [ ]:
goods_df

In [ ]:
from tqdm import trange

In [ ]:
def get_content_base(df):
    content_base = []
    goods = df.shape[1] - 1
    for i in trange(len(df)):
        content = ''
        for good_id in range(goods):
            goods_num = df.iloc[i][good_id+1]
            if goods_num > 0:
                content += (goods_df.iloc[good_id]['good'] + ' ') * goods_num
        content_base.append(content)   
    users_content = pd.DataFrame({"user_id": list(range(df.shape[0])),
                           'content': content_base})  
    return users_content

In [ ]:
users_content_train = get_content_base(train_df)

In [4]:
users_content_eval = get_content_base(eval_df)

NameError: name 'get_content_base' is not defined

In [ ]:
users_content_test = get_content_base(test_df)

In [ ]:
class Content_Based():
    def __init__(self, items):
        self.items = items
        self.content_dict = {}
        
    def avg_feature_vector(self, words, model, num_features, index2word_set):
        feature_vec = np.zeros((num_features, ), dtype='float32')
        n_words = 0
        for word in words:
            if word in index2word_set:
                n_words += 1
                feature_vec = np.add(feature_vec, model.wv[word])
        if (n_words > 0):
            feature_vec = feature_vec / n_words
        return feature_vec

    def get_items_representation(self):
        
        item_description = [gensim.utils.simple_preprocess(re.sub("[^a-zA-Zа-яА-Я]+", " ", i.lower())) 
                            for i in self.items['good']]
        #build vocabulary and train model
        self.model = gensim.models.Word2Vec(
                item_description,
                size=200,
                window=10,
                min_count=1,
                workers=10,
                iter=100)
        
        self.index2word_set = set(self.model.wv.index2word)
        self.items_embs = np.zeros((len(item_description), 200))
        for i in range(self.items_embs.shape[0]):
            self.items_embs[i] = self.avg_feature_vector(item_description[i], self.model, 200, self.index2word_set)

    def get_user_embedding(self, i, interactions):
        user_interactions = interactions.iloc[i]['content']
        print(user_interactions)
        user_processed = gensim.utils.simple_preprocess(re.sub("[^a-zA-Zа-яА-Я]+", " ", user_interactions.lower()))
        user_emb = self.avg_feature_vector(user_processed, self.model, 200, self.index2word_set)
        return user_emb

In [ ]:
CB = Content_Based(goods_df)

In [5]:
CB.get_items_representation()

NameError: name 'CB' is not defined

In [ ]:
#for i in trange()

In [ ]:
CB.get_user_embedding(6, users_content_eval).shape

In [ ]:
metrics = cosine_similarity(CB.get_user_embedding(6, users_content), CB.get_user_embedding(6, users_content_eval))